<a href="https://colab.research.google.com/github/Datasci266-Final-Project/Job-Listings/blob/main/Tokenizer%20and%20Labeler%20Notebooks/Model_2_Tokenizer_Bert_Base_Label_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Installs

!pip install pydot --quiet
!pip install transformers==4.37.2 --quiet
!pip install -U imbalanced-learn --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 5.8 MB/s eta 0:00:00


In [ ]:
#@title Imports
import pandas as pd

import numpy as np
import random
import torch

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re
from sklearn.model_selection import train_test_split

from transformers import BertTokenizer, AutoTokenizer #"FacebookAI/xlm-roberta-base"

from transformers import logging
logging.set_verbosity_error()

from collections import Counter

import pickle

In [ ]:
# Set seed to 10

seed_value = 10

random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
tf.random.set_seed(seed_value)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/DataSci 266 Project

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1V3QooLePiHR_DaZhbXQhsjmP1Ez5fv5F/DataSci 266 Project


In [ ]:
pd.set_option('display.max_colwidth', 10000)

In [ ]:
#@title import pre-split datasets
with open(r'Model Objects/train_no_oversample_df.pickle', 'rb') as handle:
    train_data = pickle.load(handle)
with open(r'Model Objects/test_data.pickle', 'rb') as handle:
    test_data = pickle.load(handle)
with open(r'Model Objects/val_data.pickle', 'rb') as handle:
    val_data = pickle.load(handle)

In [ ]:
#@title Base Salary Buckets

# Original salary buckets

# def map_salary_to_bucket(salary):
#     if pd.isnull(salary) or salary == 0:
#         return np.nan
#     elif salary < 45000:
#         return '<45k'
#     elif 45000 <= salary < 65000:
#         return '45-65k'
#     elif 65000 <= salary < 85000:
#         return '65-85k'
#     elif 85000 <= salary < 110000:
#         return '85-110k'
#     elif 110000 <= salary < 150000:
#         return '110-150k'
#     elif 150000 <= salary < 200000:
#         return '150-200k'
#     else:
#         return '>200k'


# labeled_data['salary_bucket'] = labeled_data['salary'].apply(map_salary_to_bucket)

In [ ]:
#@title Model 1 Salary Buckets

# UPDATED SALARY BUCKETS 3/23!!

# def map_salary_to_bucket(salary):
#     if pd.isnull(salary) or salary == 0:
#         return np.nan
#     elif salary < 45000:
#         return '<45k'
#     elif 45000 <= salary < 65000:
#         return '45-65k'
#     elif 65000 <= salary < 85000:
#         return '65-85k'
#     elif 85000 <= salary < 110000:
#         return '85-110k'
#     elif 110000 <= salary < 150000:
#         return '110-150k'
#     elif 150000 <= salary < 200000:
#         return '150-200k'
#     else:
#         return '>200k'


# labeled_data['salary_bucket'] = labeled_data['salary'].apply(map_salary_to_bucket)

In [ ]:
#@title Model 2 Salary Buckets

# UPDATED SALARY BUCKETS 3/29!!

def map_salary_to_bucket(salary):
    if pd.isnull(salary) or salary == 0:
        return np.nan
    elif salary < 50000:
        return '<50k'
    elif 50000 <= salary < 100000:
        return '50-100k'
    elif 100000 <= salary < 150000:
        return '100-150k'
    elif 150000 <= salary < 200000:
        return '150-200k'
    else:
        return '>200k'


train_data['salary_bucket'] = train_data['salary'].apply(map_salary_to_bucket)
test_data['salary_bucket'] = test_data['salary'].apply(map_salary_to_bucket)
val_data['salary_bucket'] = val_data['salary'].apply(map_salary_to_bucket)

In [ ]:
pd.set_option('display.max_colwidth', 10000)

In [ ]:
# Dictionaries
level_key = {
    "student_intern": 0,
    "entry": 1,
    "junior": 2,
    "mid": 3,
    "senior": 4,
    "executive": 5,
    np.nan: -1
}

# Define salary_key dictionary
# salary_key = {
#     "<45k": 0,
#     "45-65k": 1,
#     "65-85k": 2,
#     "85-110k": 3,
#     "110-150k": 4,
#     "150-200k": 5,
#     ">200k": 6,
#     np.nan: -1
# }

salary_key = {
    "<50k": 0,
    "50-100k": 1,
    "100-150k": 2,
    "150-200k": 3,
    ">200k": 4,
    np.nan: -1
}

train_data['salary_labels'] = train_data.salary_bucket.apply(lambda x: salary_key[x])

test_data['salary_labels'] = test_data.salary_bucket.apply(lambda x: salary_key[x])

val_data['salary_labels'] = val_data.salary_bucket.apply(lambda x: salary_key[x])

In [ ]:
train_data.salary_labels.value_counts()

salary_labels
-1    54925
 1     5363
 0     3187
 2     1315
 4      429
 3      401
Name: count, dtype: int64

In [ ]:
#@title Resampling - Level

#Resampling Level
X_train = train_data[['index', 'title', 'text', 'salary', 'salary_bucket', 'salary_labels', 'level_labels']]
y_train = train_data['level']

# Define the classes to undersample and oversample
levels_to_undersample = ['mid', 'junior', 'senior'] #will only undersample senior
levels_to_oversample = ['student_intern', 'executive', 'entry'] #will only oversample student_intern and 'executive'

# Create a mask to identify samples that belong to the classes to be undersampled
level_mask_undersample = np.isin(y_train, levels_to_undersample)

# Create a mask to identify samples that belong to the classes to be oversampled
level_mask_oversample = np.isin(y_train, levels_to_oversample)

level_mask_rest = ~level_mask_undersample & ~level_mask_oversample

# Perform undersampling and oversampling
rus = RandomUnderSampler(sampling_strategy='majority')
ros = RandomOverSampler()

# Undersample the 'senior' class
X_resampled_rus, y_resampled_rus = rus.fit_resample(X_train[level_mask_undersample], y_train[level_mask_undersample])

# Oversample the 'executive' and 'student_intern' classes
X_resampled_ros, y_resampled_ros = ros.fit_resample(X_train[level_mask_oversample], y_train[level_mask_oversample])

# Combine the resampled data with the original data
X_resampled = np.concatenate([X_train[level_mask_rest], X_resampled_ros, X_resampled_rus])
y_resampled = np.concatenate([y_train[level_mask_rest], y_resampled_ros, y_resampled_rus])
train_resampled = np.concatenate([X_resampled, y_resampled.reshape(-1, 1)], axis=1)

print("Original class distribution:", Counter(y_train))
print("Resampled class distribution:", Counter(train_resampled[:, -1]))

train_resampled_df = pd.DataFrame(train_resampled, columns=['index', 'title', 'text', 'salary', 'salary_bucket', 'salary_labels', 'level_labels', 'level'])



# print(train_resampled_df['salary_bucket'].value_counts())
# print(train_resampled_df['level'].value_counts())

#print(train_resampled_df.tail(10))

Original class distribution: Counter({'senior': 27340, 'entry': 9325, 'mid': 9170, 'junior': 6778, 'student_intern': 3508, 'executive': 1284, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, na

In [ ]:
# #@title Resampling - salary_buckets

# #Resampling buckets
# X_train_2 = train_resampled_df[['index', 'title', 'text', 'salary', 'salary_labels', 'level_labels', 'level']]
# y_train_2 = train_resampled_df['salary_bucket']

# # Define the classes to undersample and oversample
# buckets_to_undersample = ['85-110k', '<45k', '45-65k', '65-85k']
# buckets_to_oversample = ['110-150k', '150-200k', '>200k']

# # Create masks
# bucket_mask_undersample = np.isin(y_train_2, buckets_to_undersample)
# bucket_mask_oversample = np.isin(y_train_2, buckets_to_oversample)
# bucket_mask_rest = ~bucket_mask_undersample & ~bucket_mask_oversample

# rus = RandomUnderSampler()
# ros = RandomOverSampler()

# # Undersample the '<45k' class
# X_resampled_rus, y_resampled_rus = rus.fit_resample(X_train_2[bucket_mask_undersample], y_train_2[bucket_mask_undersample])

# # Oversample the 110-150k, 150-200k, and >200k classes
# X_resampled_ros, y_resampled_ros = ros.fit_resample(X_train_2[bucket_mask_oversample], y_train_2[bucket_mask_oversample])

# # Combine with level resampled data
# X_resampled = np.concatenate([X_train_2[bucket_mask_rest], X_resampled_ros, X_resampled_rus])
# y_resampled = np.concatenate([y_train_2[bucket_mask_rest], y_resampled_ros, y_resampled_rus])
# train_final = np.concatenate([X_resampled, y_resampled.reshape(-1, 1)], axis=1)

# print("Original class distribution:", Counter(y_train_2))
# print("Resampled class distribution:", Counter(train_final[:, -1]))

# # the columns didn't want to cooperate so I had to manually fix them.
# train_final_df = pd.DataFrame(
#     np.concatenate([X_resampled, y_resampled.reshape(-1, 1)], axis=1),
#     columns=['index', 'title', 'text', 'salary', 'salary_labels', 'level_labels', 'level', 'salary_bucket']
# )

# train_final_df = train_final_df.reindex(columns=['index', 'title', 'text', 'salary', 'salary_bucket', 'salary_labels', 'level', 'level_labels'])

# # print(train_final_df['salary_bucket'].value_counts())
# # print(train_final_df['level'].value_counts())

# #print(train_final_df.tail(10))

In [ ]:
#@title Resampling - salary_buckets

# Updated 3/29!!!

#Resampling buckets
X_train_2 = train_resampled_df[['index', 'title', 'text', 'salary', 'salary_labels', 'level_labels', 'level']]
y_train_2 = train_resampled_df['salary_bucket']

# Define the classes to undersample and oversample
buckets_to_undersample = ['<50k', '50-100k']
buckets_to_oversample = ['100-150k', '150-200k', '>200k']

# Create masks
bucket_mask_undersample = np.isin(y_train_2, buckets_to_undersample)
bucket_mask_oversample = np.isin(y_train_2, buckets_to_oversample)
bucket_mask_rest = ~bucket_mask_undersample & ~bucket_mask_oversample

rus = RandomUnderSampler()
ros = RandomOverSampler()

# Undersample the '<45k' class
X_resampled_rus, y_resampled_rus = rus.fit_resample(X_train_2[bucket_mask_undersample], y_train_2[bucket_mask_undersample])

# Oversample the 110-150k, 150-200k, and >200k classes
X_resampled_ros, y_resampled_ros = ros.fit_resample(X_train_2[bucket_mask_oversample], y_train_2[bucket_mask_oversample])

# Combine with level resampled data
X_resampled = np.concatenate([X_train_2[bucket_mask_rest], X_resampled_ros, X_resampled_rus])
y_resampled = np.concatenate([y_train_2[bucket_mask_rest], y_resampled_ros, y_resampled_rus])
train_final = np.concatenate([X_resampled, y_resampled.reshape(-1, 1)], axis=1)

print("Original class distribution:", Counter(y_train_2))
print("Resampled class distribution:", Counter(train_final[:, -1]))

# the columns didn't want to cooperate so I had to manually fix them.
train_final_df = pd.DataFrame(
    np.concatenate([X_resampled, y_resampled.reshape(-1, 1)], axis=1),
    columns=['index', 'title', 'text', 'salary', 'salary_labels', 'level_labels', 'level', 'salary_bucket']
)

train_final_df = train_final_df.reindex(columns=['index', 'title', 'text', 'salary', 'salary_bucket', 'salary_labels', 'level', 'level_labels'])

# print(train_final_df['salary_bucket'].value_counts())
# print(train_final_df['level'].value_counts())

#print(train_final_df.tail(10))

Original class distribution: Counter({nan: 48941, '50-100k': 4725, '<50k': 3255, '100-150k': 1257, '>200k': 372, '150-200k': 366})
Resampled class distribution: Counter({nan: 48941, '50-100k': 3255, '<50k': 3255, '100-150k': 1257, '>200k': 1257, '150-200k': 1257})


In [ ]:
# print(train_final_df['level_labels'].value_counts())
# print(train_final_df['level'].value_counts())
# print(train_final_df['salary_bucket'].unique())
# print(train_final_df['salary_bucket'].value_counts())
# print(train_final_df['salary_labels'].value_counts())

In [ ]:
print("Final class distributions for train dataset:")
print(train_final_df['level_labels'].value_counts())
print(train_final_df['salary_labels'].value_counts())
len(train_final_df)

Final class distributions for train dataset:
level_labels
1     9310
0     9310
5     9284
3     9152
-1    8627
4     6786
2     6753
Name: count, dtype: int64
salary_labels
-1    48941
0      3255
1      3255
2      1257
3      1257
4      1257
Name: count, dtype: int64


59222

In [ ]:
#@title BERT

checkpoint = 'bert-base-cased'
bert_tokenizer = BertTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
#@title Tokenize
#ONLY need to tokenize train data, since that was re under/over sampled

texts_train = train_final_df['text'].tolist()

MAX_SEQUENCE_LENGTH = 512

bert_train_tokenized = bert_tokenizer(
    texts_train,
    max_length=MAX_SEQUENCE_LENGTH,
    truncation=True,
    padding='max_length',
    return_tensors='tf'
)

bert_train_inputs = [bert_train_tokenized.input_ids,
                     bert_train_tokenized.token_type_ids,
                     bert_train_tokenized.attention_mask]



In [ ]:
train_final_df['level_labels'] = train_final_df['level_labels'].astype(int)
train_final_df['salary_labels'] = train_final_df['salary_labels'].astype(int)

In [ ]:
#@title Export labels, tokenized train inputs
# ONLY need to export updated train inputs and dataset
# Need to export ALL updated train,val,test labels since buckets were adjusted


#export tokenized inputs
with open('model_2_train_inputs_bert.pickle', 'wb') as handle:
    pickle.dump(bert_train_inputs, handle, protocol=pickle.HIGHEST_PROTOCOL)



# export train, val, test
with open('model_2_train_final_df.pickle', 'wb') as handle:
    pickle.dump(train_final_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('model_2_test_data.pickle', 'wb') as handle:
    pickle.dump(test_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('model_2_val_data.pickle', 'wb') as handle:
    pickle.dump(val_data, handle, protocol=pickle.HIGHEST_PROTOCOL)



# export labels
with open('model_2_labels_level_train.pickle', 'wb') as handle:
    pickle.dump(train_final_df['level_labels'], handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('model_2_labels_salary_train.pickle', 'wb') as handle:
    pickle.dump(train_final_df['salary_labels'], handle, protocol=pickle.HIGHEST_PROTOCOL)


with open('model_2_labels_level_test.pickle', 'wb') as handle:
    pickle.dump(test_data['level_labels'], handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('model_2_labels_salary_test.pickle', 'wb') as handle:
    pickle.dump(test_data['salary_labels'], handle, protocol=pickle.HIGHEST_PROTOCOL)


with open('model_2_labels_level_val.pickle', 'wb') as handle:
    pickle.dump(val_data['level_labels'], handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('model_2_labels_salary_val.pickle', 'wb') as handle:
    pickle.dump(val_data['salary_labels'], handle, protocol=pickle.HIGHEST_PROTOCOL)